In [3]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare

In [4]:
draftdf = pd.read_csv('draft_data.csv')

In [5]:
performancedf = pd.read_csv('performance_data.csv')

In [6]:
draftdf.head()

,Unnamed: 0,Round,Pick,Overall,Team,Player,Position,Year,overall_pick,pick_points
0,0,1.0,1.0,2018-1,BUF,Rasmus Dahlin,D,2018,1,216
1,1,1.0,2.0,2018-2,CAR,Andrei Svechnikov,F,2018,2,215
2,2,1.0,3.0,2018-3,MTL,Jesperi Kotkaniemi,F,2018,3,214
3,3,1.0,4.0,2018-4,OTT,Brady Tkachuk,F,2018,4,213
4,4,1.0,5.0,2018-5,ARI,Barrett Hayton,F,2018,5,212


In [7]:
performancedf.head()

,Unnamed: 0,Year,Team,GP,W,L,year_diff,wins_change
0,232,2007,ANA,82,48,20,NaN,NaN
1,267,2008,ANA,82,47,27,1.0,-0.020833
2,313,2009,ANA,82,42,33,1.0,-0.106383
3,345,2010,ANA,82,39,32,1.0,-0.071429
4,372,2011,ANA,82,47,30,1.0,0.205128


In [13]:
draftdf['round_mult'] = 8 - draftdf['Round']

In [19]:
draftdf['pick_points'] = draftdf['round_mult'] * (39 - draftdf['Pick'])

In [22]:
performancedf['change_direction'] = 'positive'

In [24]:
mask = performancedf.wins_change < 0
performancedf.loc[mask, 'change_direction'] = 'negative'

In [25]:
performancedf.head()

,Unnamed: 0,Year,Team,GP,W,L,year_diff,wins_change,change_direction
0,232,2007,ANA,82,48,20,NaN,NaN,positive
1,267,2008,ANA,82,47,27,1.0,-0.020833,negative
2,313,2009,ANA,82,42,33,1.0,-0.106383,negative
3,345,2010,ANA,82,39,32,1.0,-0.071429,negative
4,372,2011,ANA,82,47,30,1.0,0.205128,positive
